In [ ]:
stream_name = 'test'

In [ ]:
import pandas as pd

df = pd.DataFrame(columns=['timestamp', 'platform', 'username', 'message'])

In [ ]:
import re
import pandas as pd

data = []

# Define regex patterns for parsing
TWITCH_LOG_PATTERN = re.compile(r'^(\d{4}-\d{2}-\d{2}_\d{2}:\d{2}:\d{2}) — :([^!]+)!.* PRIVMSG #\w+ :(.*)$')
YOUTUBE_LOG_PATTERN = re.compile(r'^(\d{4}-\d{2}-\d{2}_\d{2}:\d{2}:\d{2}) — \[(.*?)\]- (.*)$')

for platform in ['twitch', 'youtube']:
    log_file = f'logs/{stream_name}_{platform}_live_chat.log'
    try:
        with open(log_file, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue

                timestamp, username, message = None, None, None

                if platform == 'twitch':
                    match = TWITCH_LOG_PATTERN.match(line)
                    if match:
                        timestamp, username, message = match.groups()

                elif platform == 'youtube':
                    match = YOUTUBE_LOG_PATTERN.match(line)
                    if match:
                        timestamp, username, message = match.groups()

                        if username and username.startswith('@'):
                            username = username[1:]

                if timestamp and username and message:
                    data.append({
                        'timestamp': timestamp,
                        'platform': platform,
                        'stream': stream_name,
                        'username': username,
                        'message': message
                    })

    except FileNotFoundError:
        print(f'Log file not found: {log_file}')

if data:
    new_df = pd.DataFrame(data)
    df = pd.concat([df, new_df], ignore_index=True)

# Convert timestamp to datetime object for better manipulation
if not df.empty:df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d_%H:%M:%S')

print(f'Parsed {len(df)} messages.')

In [ ]:
df

In [ ]:
df.to_csv(f'{stream_name}_chat_data.csv', index=False)